<a href="https://colab.research.google.com/github/ThirumalaRaoModepalli/demo_Docker/blob/main/Codes/Copy_Gsheet_Formatting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gspread
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Authenticate with your Google Sheets account
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file('/content/api_key.json', scopes = scope)
gc = gspread.Client(auth = creds)

In [9]:
!pip install -qq gspread-formatting --upgrade

In [4]:
from gspread_formatting import *

In [50]:
spreadsheet = gc.open('Copy of about_Us_Links')
worksheet = spreadsheet.worksheet('Sheet1')

In [51]:
def formatting(worksheet):

    all_values = worksheet.get_all_values()

    # Calculate the number of filled rows and columns
    num_filled_rows = len(all_values)
    num_filled_columns = len(all_values[0]) if num_filled_rows > 0 else 0
    cell_range = f"A2:{chr(ord('A') + num_filled_columns - 1)}{num_filled_rows + 1}"
    header_fmt = cellFormat(
        backgroundColor = color(1, 0.8, 1),
        textFormat = textFormat(bold=True, foregroundColor = color(0, 0, 0)),
        horizontalAlignment = 'CENTER',
        borders = borders(
            top = border('SOLID', color = color(0, 0, 0)),
            bottom = border('SOLID', color = color(0, 0, 0)),
            left = border('SOLID', color = color(0, 0, 0)),
            right = border('SOLID', color = color(0, 0, 0))
        )
    )

    data_fmt = cellFormat(
        horizontalAlignment = 'LEFT',
        borders = borders(
            top = border('SOLID', color = color(0, 0, 0)),
            bottom = border('SOLID', color = color(0, 0, 0)),
            left = border('SOLID', color = color(0, 0, 0)),
            right = border('SOLID', color = color(0, 0, 0))
        )
    )
    headers = worksheet.row_values(1)  # Get the headers from the first row
    num_columns = len(headers)
    max_lengths = []

    for i in range(num_columns):
        column_values = worksheet.col_values(i + 1)  # Google Sheets columns are 1-indexed
        length = max(len(value) for value in column_values)
        max_length = 250 if length>=40 else 140
        max_lengths.append((chr(65 + i), max_length))
    set_column_widths(worksheet,max_lengths)
    worksheet.format("A:ZZ", {"wrapStrategy": "WRAP"})
    cell_format = {
        "verticalAlignment": "TOP"
    }
    worksheet.format('A1:ZZ1000', cell_format)
    format_cell_range(worksheet, '1', header_fmt)

    # Format the data rows
    format_cell_range(worksheet, cell_range, data_fmt)

    # Freeze the header row
    set_frozen(worksheet, rows=1)

    print("Successfully made changes in the Google sheet.")

### Function to Delete Additional Rows and Columns

In [62]:
def trim_Sheet(worksheet):
    service = build('sheets', 'v4', credentials = creds)

    spreadsheet_id = spreadsheet.id
    sheet_id = worksheet.id

    all_values = worksheet.get_all_values()
    num_filled_rows = len(all_values)
    num_filled_columns = len(all_values[0]) if num_filled_rows > 0 else 0

    # Deleting Rows
    requests = [
        {'deleteDimension': {
            'range':   {
                        'sheetId': sheet_id,
                        'dimension': 'ROWS',
                        'startIndex': num_filled_rows, # 0-Based Indexing
                        }
                }
        }
    ]

    # Deleting Columns
    requests.append(
        {
            'deleteDimension': {
              'range':  {
                          'sheetId': sheet_id,
                          'dimension': 'COLUMNS',
                          'startIndex': num_filled_columns, # 0-Based Indexing
                        }
                }
        }
    )

    # Execute the batch update request
    body = { 'requests' : requests }

    try:
        response = service.spreadsheets().batchUpdate(spreadsheetId = spreadsheet_id, body = body).execute()
        print('Unused rows and columns removed successfully.')
    except HttpError as e:
        error_message = e._get_reason()
        if "Cannot delete a row that doesn't exist" in error_message:
            print("One or more specified rows  do not exist.")
        elif "Cannot delete a column that doesn't exist" in error_message:
            print("One or more specified columns do not exist.")
        else:
            print(e)

In [63]:
formatting(worksheet)
trim_Sheet(worksheet)

Successfully made changes in the Google sheet.
One or more specified rows  do not exist.
